# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [44]:
#pip install gmaps


In [45]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import pprint

# Import API key
from api_keys import g_key
gmaps.configure(api_key = g_key)

In [73]:
#output data file (csv)
cities = pd.read_csv("output_data/cities.csv")
cities

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [111]:
# Configure gmaps

gmaps.configure(api_key=g_key)

# Store lat and lng in locations
locations = cities[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = cities["Humidity"].astype(float)
maxhumidity = humidity.max

In [112]:
#Adding heatmap layer to map

fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

#Create and add heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
fig.add_layer(heat_layer)
#Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [113]:
#Narrow down DataFrame to find ideal weather condition
#temparature between 60 and 75, WindSpeed between 1 and 5
#cloudiness equals zero
narrowed_cities_df = cities.loc[(cities["Max Temp"] > 60) &
                                 (cities["Max Temp"] < 75) & 
                                  (cities["Wind Speed"] > 1) & 
                                  (cities["Wind Speed"] < 5) & 
                                  (cities["Cloudiness"] == 0), :]
narrowed_cities_df = narrowed_cities_df.dropna(how='any')
narrowed_cities_df.reset_index(inplace=True)
del narrowed_cities_df['index']
narrowed_cities_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,141,sanchazi,0,CN,1558378781,82,30.43,103.62,64.40,2.24
1,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
2,508,noumea,0,NC,1558378811,77,-22.28,166.46,64.40,2.24
3,538,la seyne-sur-mer,0,FR,1558378897,72,43.10,5.88,69.01,2.24


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [123]:
#Create a hotel_df
hotel_df = narrowed_cities_df.loc[:,["City","Country", "Lat", "Lng"]]

# insert a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""


hotel_df


,City,Country,Lat,Lng,Hotel Name
0,sanchazi,CN,30.43,103.62,
1,mogok,MM,22.92,96.51,
2,noumea,NC,-22.28,166.46,
3,la seyne-sur-mer,FR,43.10,5.88,


In [124]:
#Create a list to for hotel names  to place in the DataFrame
hotellist = []

for i in range(len(narrowed_cities_df)):
    lat = narrowed_cities_df.loc[i]['Lat']
    lng = narrowed_cities_df.loc[i]['Lng']

    #insert keyword params to dict
    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    #query url and make API request
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotellist.append(jsn['results'][0]['name'])
    
    #Save hotel name to dataframe
    
    except:
        hotellist.append("")
narrowed_cities_df["Hotel Name"] = hotellist
narrowed_cities_df = narrowed_cities_df.dropna(how='any')
narrowed_cities_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,141,sanchazi,0,CN,1558378781,82,30.43,103.62,64.40,2.24,Chengdu
1,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,Mogok
2,508,noumea,0,NC,1558378811,77,-22.28,166.46,64.40,2.24,Noumea
3,538,la seyne-sur-mer,0,FR,1558378897,72,43.10,5.88,69.01,2.24,Toulon


In [125]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [126]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))